<a href="https://colab.research.google.com/github/charoo-rumsan/DSPy_research/blob/main/DSPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install dspy


Configure your LLM

In [17]:
import dspy

llm = dspy.LM('ollama_chat/llama3.2-vision:latest',
  api_base='http://localhost:11434',
  api_key='', temperature = 0.3)
dspy.configure(lm=llm)

Create a Signature

A signature defines input and output fields

In [18]:
class EmotionSignature(dspy.Signature):
    sentence = dspy.InputField()
    emotion = dspy.OutputField(desc="happy or sad")


Create a Module (worker)

In [19]:
class EmotionClassifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(EmotionSignature)

    def forward(self, sentence):
        result = self.predict(sentence=sentence)
        return result.emotion


Create Train Examples (devset)

These are tiny examples DSPy will use to score and improve the module.

In [20]:
train_examples = [
    dspy.Example(sentence="I love sunny days!", emotion="happy").with_inputs("sentence"),
    dspy.Example(sentence="This is the worst day ever.", emotion="sad").with_inputs("sentence"),
    dspy.Example(sentence="I got a new puppy!", emotion="happy").with_inputs("sentence"),
    dspy.Example(sentence="I feel lonely today.", emotion="sad").with_inputs("sentence")
]

Create a Program

This is what you will run after training

In [21]:
classifier = EmotionClassifier()


Define a Metric

We use simple accuracy: correct = 1, wrong = 0

In [22]:
def simple_accuracy(example, prediction):
    return int(prediction == example.emotion)


Run Evaluation BEFORE optimization

In [23]:
before = dspy.Evaluate(
    devset=train_examples,
    metric=simple_accuracy
)

print("Accuracy before optimization:", before(classifier))

2025/11/25 06:24:01 ERROR dspy.utils.parallelizer: Error for Example({'sentence': 'I love sunny days!', 'emotion': 'happy'}) (input_keys={'sentence'}): litellm.APIConnectionError: Ollama_chatException - [Errno 111] Connection refused. Set `provide_traceback=True` for traceback.
2025/11/25 06:24:02 ERROR dspy.utils.parallelizer: Error for Example({'sentence': 'This is the worst day ever.', 'emotion': 'sad'}) (input_keys={'sentence'}): litellm.APIConnectionError: Ollama_chatException - [Errno 111] Connection refused. Set `provide_traceback=True` for traceback.
2025/11/25 06:24:02 ERROR dspy.utils.parallelizer: Error for Example({'sentence': 'I feel lonely today.', 'emotion': 'sad'}) (input_keys={'sentence'}): litellm.APIConnectionError: Ollama_chatException - [Errno 111] Connection refused. Set `provide_traceback=True` for traceback.
2025/11/25 06:24:02 ERROR dspy.utils.parallelizer: Error for Example({'sentence': 'I got a new puppy!', 'emotion': 'happy'}) (input_keys={'sentence'}): lite

Accuracy before optimization: EvaluationResult(score=0.0, results=<list of 4 results>)


Choose an Optimizer

BootstrapFewShot is simplest and works well

In [31]:
optimizer = dspy.BootstrapFewShot(max_bootstrapped_demos=3)

Optimize the program

DSPy will automatically improve prompts & examples.

In [33]:
optimized_classifier = optimizer.compile(classifier, trainset=train_examples)

100%|██████████| 4/4 [00:24<00:00,  6.19s/it]

Bootstrapped 0 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.


Evaluate AFTER optimization

In [35]:
after = dspy.Evaluate(
    devset=train_examples,
    metric=simple_accuracy
)

print("Accuracy after optimization:", after(optimized_classifier))

2025/11/25 06:29:32 ERROR dspy.utils.parallelizer: Error for Example({'sentence': 'I got a new puppy!', 'emotion': 'happy'}) (input_keys={'sentence'}): litellm.APIConnectionError: Ollama_chatException - [Errno 111] Connection refused. Set `provide_traceback=True` for traceback.
2025/11/25 06:29:32 ERROR dspy.utils.parallelizer: Error for Example({'sentence': 'I feel lonely today.', 'emotion': 'sad'}) (input_keys={'sentence'}): litellm.APIConnectionError: Ollama_chatException - [Errno 111] Connection refused. Set `provide_traceback=True` for traceback.
2025/11/25 06:29:32 ERROR dspy.utils.parallelizer: Error for Example({'sentence': 'I love sunny days!', 'emotion': 'happy'}) (input_keys={'sentence'}): litellm.APIConnectionError: Ollama_chatException - [Errno 111] Connection refused. Set `provide_traceback=True` for traceback.
2025/11/25 06:29:32 ERROR dspy.utils.parallelizer: Error for Example({'sentence': 'This is the worst day ever.', 'emotion': 'sad'}) (input_keys={'sentence'}): lite

Accuracy after optimization: EvaluationResult(score=0.0, results=<list of 4 results>)


Test your improved classifier

In [1]:
print(optimized_classifier("I just won the lottery!"))
print(optimized_classifier("Everything is going wrong today."))


NameError: name 'optimized_classifier' is not defined